In [1]:
import sympy as sp
import numpy as np
from scipy.linalg import null_space
import sklearn as sl
import math

In [2]:
N = 227179
B = 50

#B = math.ceil(math.e**(.5*(math.log(N)*math.log(math.log(N)))**.5))
#print(B)

In [3]:
#solve for n in (sqrt(N) + n)^2 - N = 0(mod p)

def nSolve(n, p):
    root = math.ceil(n**.5)
    r = []
    for i in range(p):
        if ((root + i)**2 - N) % p == 0:
            r.append(i)
    return r

In [4]:
#creates the factor base of N up to B

def eulersCriterion(n, p):
    n = n % p
    for i in range(2, p, 1):
        if ((i * i) % p == n):
            return True
    return False

def factorBase(n, b):

    prime = [True for i in range(n+1)] 

    p = 2
    while(p * p <= n):
        if (prime[p] == True):
            
            for i in range(p * p, n + 1, p):
                prime[i] = False
        p += 1
    
    c = []
    for p in range(2, b):

        if prime[p] and eulersCriterion(n, p):
            c.append(p)
            
    if 2 not in c: c.insert(0, 2)
    return c

print(factorBase(N, B))

[2, 3, 5, 7, 13, 17, 23, 29, 37, 41, 43, 47]


In [5]:
#find smooth numbers using Tonelli–Shanks algorithm and returns matrix with rows as smooth numbers
#INPUT: large integer n, bound b
#OUPUT: smooth number sequence, matrix of exponentials of smooth numbers, reduced matrix

#these variables are subject to optimization
maxSeq = 20 #size of number sequence collected using x^2 - N
maxPow = 3 #max power of the factor base primes

def createMat(n, b):
    
    root = math.ceil(n**.5)
    fb = factorBase(n, b)

    #number sequence
    a = []
    for i in range(maxSeq):
        a.append((root + i)**2 - n)
    #print(a)

    #exponent matrix (mat[row][col])
    rows = len(a)
    cols = len(fb)
    reduced_mat = [[0 for _ in range(cols)] for _ in range(rows)]
    raw_mat = [[0 for _ in range(cols)] for _ in range(rows)]
    #print(mat)

    #initial matrix
    for i in range(1, maxPow + 1): #power of factor base prime
        for p in range(len(fb)): #prime
            #print(nSolve(p**i), p**i)

            for j in nSolve(n, fb[p]**i): #starting sequence number
                for k in range(j, maxSeq, fb[p]**i): #sequence number
                    a[k] = a[k] / fb[p]
                    if k < rows and p < cols: 
                        reduced_mat[k][p] = (reduced_mat[k][p] - 1)*-1
                        raw_mat[k][p] = raw_mat[k][p] + 1
            #print(a)
            
    #print(a)
    #print(reduced_mat)
    #print(raw_mat)
    
    smoothRoots = [] #smooth number roots
    for i in range(len(a)):
        if a[i] == 1: smoothRoots.append(i) 
    
    rawM = [] #smooth number exp matrix (raw)
    for i in range(len(a)):
        if a[i] == 1: rawM.append(raw_mat[i]) 

    redM = [] #smooth number exp matrix (reduced)
    for i in range(len(a)):
        if a[i] == 1: redM.append(reduced_mat[i])      
    
    smoothRoots = np.array(smoothRoots)
    rawM = np.array(rawM)
    redM = np.array(redM)
    #print(smoothRoots)
    #print(rawM)
    #print(redM)
    #print(redM.shape)
    
    return [smoothRoots, rawM, redM]

createMat(N, B)

[array([ 0,  1,  2,  4,  5,  6, 10, 11, 16, 19]),
 array([[1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [0, 1, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
        [1, 3, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0],
        [1, 1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0],
        [0, 2, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0]]),
 array([[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
        [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0]])]

In [6]:
#finds dependence relations of matrix
#INPUT: smooth matrix
#OUPUT: list with elements as indices of 

test = np.array([[1,1,0,0],[1,1,0,1],[0,1,1,1],[0,0,1,0],[0,0,0,1]])
#print(test)

def findDep(mat):
    
    '''A Fast Algorithm for
        Gaussian Elimination over GF (2) and its Implementation on the
        GAPP.' Journal of Parallel and Distributed Computing 13.1
        (1991): 118-122.'''
    
    pRows = np.zeros(mat.shape[0])
    for i in range(mat.shape[1]):
        
        if (sum(mat[:, i]) != 0):
            
            p = list(mat[:, i]).index(1)
            pRows[p] = 1
            
            for j in range(mat.shape[1]):
                
                if (j == i):
                    continue
                    
                if (mat[p][j] == 1):
                    mat[:, j] = (mat[:, j] + mat[:, i]) % 2
    #print(mat)
    #print(pRows)
    #print(list(pRows).index(0))
    
    #returns indices equaling 1 of the first nonpivot row of the matrix
    nonP = list(pRows).index(0)
    nonPInd = []
    for r in range(mat.shape[1]):
        if mat[nonP][r] == 1:
            nonPInd.append(r)
    #print(nonPInd)
    
    nullSpace = [nonP]
    for s in range(mat.shape[0]):
        if (sum(mat[s, :]) == 1) and (list(mat[s, :]).index(1) in nonPInd):
            nullSpace.append(s)

    
    #print(nullSpace)
    
    return nullSpace

findDep(createMat(N, B)[2])

[8, 0, 4]

In [7]:
#MAIN

x = 1
y = 1
root = math.ceil(N**.5)
sol = findDep(createMat(N, B)[2])

for i in sol:
    x = x * (root + createMat(N, B)[0][i])

x = x % N
#print(x)

expm = createMat(N, B)[1]
primes = factorBase(N, B)
#print(createMat(N, B)[1].shape[1])

expSum = np.zeros(expm.shape[1])
for i in sol:
    expSum = expSum + expm[i]

expSum = expSum / 2

for i in range(len(primes)):
    y = y * int(primes[i] ** expSum[i])

#print(y)


factor1 = math.gcd(x - y, N)
factor2 = int(N / factor1)
print(factor1, factor2)

1447 157
